In [1]:
import clustertools as ctools
import numpy as np

# Loading and Advancing

# Loading


To manually load a snapshot of a cluster, simply read in the file via your preferred method, declare a StarCluster with the appropriate units and origin, and add stars. For example, consider a snapshot of a star cluster in a file named 00000.dat with columns of stellar mass, position (x,y,z) in pc , and velocity (vx,vy,vz) in km/s. The origin of the system is the cluster's centre. A StarCluster can be initialized via:

In [2]:
m,x,y,z,vx,vy,vz=np.loadtxt('00000.dat',unpack=True)
cluster=ctools.StarCluster(units='pckms',origin='cluster')
cluster.add_stars(x,y,z,vx,vy,vz,m)

When manually setting up a cluster, key cluster parameters will not be calculated unless the function ``key_params()`` is called. Similarly, calculations that require the stars to be ordered by distance from the cluster's centre are not calculated unless one sets the ``do_order`` flag. 

In [3]:
cluster.key_params(do_order=True)

In [4]:
print('Total Number of Stars = ',cluster.ntot)
print('Total Mass = ',cluster.mtot)
print('Mean Mass = ',cluster.mmean)
print('Mean Radius = ',cluster.rmean)
print('Maximum Radius = ',cluster.rmax)
print('Half-mass radius = ',cluster.rm)
print('Projected Half-mass radius = ',cluster.rmpro)
print('10% Lagrange radius = ',cluster.r10)
print('Projected 10% Lagrange radius = ',cluster.r10pro)

Total Number of Stars =  1000
Total Mass =  349.28177634
Mean Mass =  0.34928177634
Mean Radius =  2.45102483757
Maximum Radius =  8.32425911109
Half-mass radius =  1.96178691294
Projected Half-mass radius =  1.50302785137
10% Lagrange radius =  0.733265989776
Projected 10% Lagrange radius =  0.490977522097


Alternatively, several functions have been written to more easily load a snapshot of a cluster.  For example, snapshot 00000.dat could more easily be loaded via:

In [5]:
cluster=ctools.load_cluster('snapshot',filename='00000.dat',units='pckms',origin='cluster')

When using load_cluster, key cluster parameters are instantly calculated.

In [6]:
print('Total Number of Stars = ',cluster.ntot)
print('Total Mass = ',cluster.mtot)
print('Mean Mass = ',cluster.mmean)
print('Mean Radius = ',cluster.rmean)
print('Maximum Radius = ',cluster.rmax)
print('Half-mass radius = ',cluster.rm)
print('Projected Half-mass radius = ',cluster.rmpro)
print('10% Lagrange radius = ',cluster.r10)
print('Projected 10% Lagrange radius = ',cluster.r10pro)


Total Number of Stars =  1000
Total Mass =  349.28177634
Mean Mass =  0.34928177634
Mean Radius =  2.46448430701
Maximum Radius =  8.47641756979
Half-mass radius =  1.94950947706
Projected Half-mass radius =  1.51415254024
10% Lagrange radius =  0.700050232243
Projected 10% Lagrange radius =  0.451505659588


If the columns in your snapshot do not have the format mass,x,y,z,vx,vy,vz,i_d,kw, then use ``col_names`` and ``col_nums`` when using ``load_cluster``

In [7]:
cluster=ctools.load_cluster('snapshot',filename='00000.dat',units='pckms',origin='cluster',
                            col_names=["m", "x", "y", "z", "vx", "vy", "vz"],col_nums=[0, 1, 2, 3, 4, 5, 6],)

If the orbital information of your simulated cluster is known, and is contained in a file with columns of time, x, y, z, vx, vy, vz then either the filename can be given to the ``load_cluster`` command using the ``ofilename`` flag or it can be read in and added separately. If the units in ``orbit.dat`` are not the same as ``cluster.units``, one can use the ``ounits`` flag in ``add_orbit``. They will be converted to cluster.units.

In [8]:
t,xgc,ygc,zgc,vxgc,vygc,vzgc=np.loadtxt('orbit.dat',unpack=True)
cluster.tphys=t
cluster.add_orbit(xgc,ygc,zgc,vxgc,vygc,vzgc,ounits='kpckms')
print(cluster.tphys,cluster.xgc,cluster.ygc,cluster.zgc,cluster.vxgc,cluster.vygc,cluster.vzgc)

[  0.  10.] [ 10000.      0.] [     0.  10000.] [ 0.  0.] [   0.  220.] [ 220.    0.] [ 0.  0.]


Functions have also been written to automatically load snapshots from the commonly used codes NBODY6 and gyrfalcon. To load an NBODY6 snapshot from OUT34 and OUT9:

In [9]:
cluster=ctools.load_cluster('nbody6')

FileNotFoundError: [Errno 2] No such file or directory: './OUT34'

``clustertools`` knows that when reading in NBODY6 snapshots, the units will be ``'nbody'`` units and the origin will be ``'cluster'``.

For an NBODY6 simulation with stellar evolution, the first snapshot can be read in from fort.82 and fort.83 via:

In [ ]:
cluster=ctools.load_cluster('nbody6se')

Finally, for a gyrfalcon simulation. the first cluster snapshot can be read in using:

In [ ]:
cluster=ctools.load_cluster('gyrfalcon',filename='cluster.nemo.dat')

Note that since gyrfalcon does not have a default output filename like NBODY6, the filename must be specifified. Furthermore, gyrfalcon's output is a binary file that must be converted to ascii using NEMO's ``s2a`` command before it can be read in by ``clustertools``.

# Advancing

In most cases, it is desirable to read more than just the first timestep of a simulation. ``clustertools`` has been set up to easily read in a snapshot, calculate key cluster parameters, and then advance to the next snapshot. An advance can easily be done using:

In [ ]:
cluster=ctools.load_cluster('snapshot',filename='00000.dat',units='pckms',origin='cluster')
print(cluster.rm,cluster.mtot)
cluster=ctools.advance_cluster(cluster)
print(cluster.rm,cluster.mtot)


For the case above where the first snapshot had a filename of 00000.dat, ``advance_cluster`` will look for a filename titled 00001.dat. If your filenames have a different naming convention, just specify some of the keyword arguments when loading the first snapshot. For example, if your snapshots are named snap_cluster_001.txt and snap_cluster_002.txt, they can be loaded via:

In [ ]:
cluster=ctools.load_cluster('snapshot',nsnap=1,nzfill=3,snapbase='snap_cluster_',snapend='.txt',units='pckms',origin='cluster')
print(cluster.rm,cluster.mtot)
cluster=ctools.advance_cluster(cluster)
print(cluster.rm,cluster.mtot)
